# Серия тиков с разной ценой, количество тиков N, расстояние во времени меньше T миллисекунд

In [1]:
from simple.ohlc import debounce, npJoin
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestMarket, getProfit
import numpy as np
from simple.numpy import rolling_apply

In [2]:
threshold = 2_000_000  # default resolution in microseconds
period = 30

In [3]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
T[-5:]

rec.array([('2022-05-08T00:59:59.571000', 95815, 2617.99,   -2.),
           ('2022-05-08T00:59:59.584000', 95816, 2617.99,   -2.),
           ('2022-05-08T00:59:59.596000', 95817, 2617.99,   -2.),
           ('2022-05-08T00:59:59.706000', 95818, 2618.  ,   32.),
           ('2022-05-08T00:59:59.826000', 95819, 2618.  , 1912.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]

In [5]:
C = debounce(T)
C[-5:]

rec.array([('2022-05-08T00:59:57.402000', 12593, 2618.405,       0, 1.3980e+03, 1398.,  0.0000e+00,  2,  2,  0),
           ('2022-05-08T00:59:57.418999', 12594, 2618.435,       0, 1.9920e+03, 1992.,  0.0000e+00,  1,  1,  0),
           ('2022-05-08T00:59:58.848000', 12595, 2618.165, 1429001, 6.0580e+03, 2123., -3.9350e+03, 46, 19, 27),
           ('2022-05-08T00:59:58.848000', 12596, 2618.115,       0, 3.0000e+00,    0., -3.0000e+00,  1,  0,  1),
           ('1970-01-01T00:00:00.000000', 12597, 2618.015,  978000, 3.3762e+04, 2243., -3.1519e+04, 27,  5, 22)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i8'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i8'), ('BuyCount', '<i8'), ('SellCount', '<i8')])

In [6]:
len(T), len(C)

(95820, 12598)

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T00:59:57.149000', 12590, 2618.425,  819000, -9.3230e+03, 1092., -1.0415e+04, 84,  5, 79),
           ('2022-05-08T00:59:57.186999', 12591, 2618.285,   22999, -1.1100e+02,  410., -5.2100e+02,  8,  2,  6),
           ('2022-05-08T00:59:58.848000', 12595, 2618.165, 1429001, -1.8120e+03, 2123., -3.9350e+03, 46, 19, 27),
           ('2022-05-08T00:59:58.848000', 12596, 2618.115,       0, -3.0000e+00,    0., -3.0000e+00,  1,  0,  1),
           ('1970-01-01T00:00:00.000000', 12597, 2618.015,  978000, -2.9276e+04, 2243., -3.1519e+04, 27,  5, 22)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i8'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i8'), ('BuyCount', '<i8'), ('SellCount', '<i8')])

In [8]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [9]:
K = npJoin(C.DT, S.DT)
len(K)

12598

In [10]:
# calculate sum of duration/volume by sliding window
def getSum(x):
    return np.sum(x).astype(int)


DurationA = rolling_apply(getSum, period, C.Duration)
VolumeA = rolling_apply(getSum, period, C.Volume)
len(DurationA)

12598

In [11]:
# trade signal
signal = 0
SignalA = np.zeros_like(DurationA)
for i in range(len(SignalA)):
    if DurationA[i] < threshold and VolumeA[i] > 0:
        signal = 1
    elif DurationA[i] < threshold and C.Volume[i] < 0:
        signal = -1
    SignalA[i] = signal

SignalA[-10:]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [12]:
D = npBacktestMarket(S.DT[K], S.A[K], S.B[K], SignalA, 0)
D[-5:]

rec.array([(11962, '2022-05-08T00:57:41.550000', 2616.44995117, 2616.44482422, 12011, '2022-05-08T00:57:42.198000', 2616.60009766, 2616.66015625,  1.),
           (12011, '2022-05-08T00:57:42.198000', 2616.60009766, 2616.66015625, 12192, '2022-05-08T00:58:43.629000', 2617.64990234, 2617.54492188, -1.),
           (12192, '2022-05-08T00:58:43.629000', 2617.64990234, 2617.54492188, 12327, '2022-05-08T00:59:33.395000', 2615.4699707 , 2615.625     ,  1.),
           (12327, '2022-05-08T00:59:33.395000', 2615.4699707 , 2615.625     , 12397, '2022-05-08T00:59:34.078000', 2615.11010742, 2615.08007812, -1.),
           (12397, '2022-05-08T00:59:34.078000', 2615.11010742, 2615.08007812, 12548, '2022-05-08T00:59:41.940000', 2617.55004883, 2617.60986328,  1.)],
          dtype=[('X0', '<i8'), ('T0', '<M8[us]'), ('Price0', '<f8'), ('MidPrice0', '<f8'), ('X1', '<i8'), ('T1', '<M8[us]'), ('Price1', '<f8'), ('MidPrice1', '<f8'), ('Size', '<f8')])

In [13]:
Long = D[D.Size > 0]
Short = D[D.Size < 0]

P = getProfit(D)
P[-5:]

rec.array([(12011, '2022-05-08T00:57:42.198000',  0.15014648,  0.21533203, 2.61652502, -2.46637854),
           (12192, '2022-05-08T00:58:43.629000', -1.04980469, -0.88476562, 2.617125  , -3.66692969),
           (12327, '2022-05-08T00:59:33.395000', -2.17993164, -1.91992188, 2.61655994, -4.79649158),
           (12397, '2022-05-08T00:59:34.078000',  0.35986328,  0.54492188, 2.61529004, -2.25542676),
           (12548, '2022-05-08T00:59:41.940000',  2.43994141,  2.52978516, 2.61633008, -0.17638867)],
          dtype=[('Index', '<i8'), ('DT', '<M8[us]'), ('RawPnL', '<f8'), ('MidPnL', '<f8'), ('Fee', '<f8'), ('Profit', '<f8')])

In [14]:
# indexed chart with bidask
fig = chartFigure(height=800, rows=4,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.25),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.25),
    Bid=dict(x=C.Index, y=S.B[K], color='green', opacity=0.2),
    Ask=dict(x=C.Index, y=S.A[K], color='red', opacity=0.2),

    # trades
    EnterLong=dict(x=Long.X0, y=Long.Price0, mode='markers',
                   marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green')),
    ExitLong=dict(x=Long.X1, y=Long.Price1, mode='markers',
                  marker=dict(symbol='x', size=10, line_color='darkgreen', line_width=1, color='green', opacity=0.4)),

    EnterShort=dict(x=Short.X0, y=Short.Price0, mode='markers',
                    marker=dict(symbol='triangle-down', size=12, line_color='darkred', line_width=1, color='red')),
    ExitShort=dict(x=Short.X1, y=Short.Price1, mode='markers',
                   marker=dict(symbol='x', size=10, line_color='darkred', line_width=1, color='red', opacity=0.4)),

    # Signal and duration indicators
    Signal=dict(x=C.Index, y=SignalA, color='magenta', row=3, shape='hv'),
    Duration=dict(x=C.Index, y=DurationA, color='blue', row=4),

    # Profit
    RawPnL=dict(x=P.Index, y=P.RawPnL.cumsum(), secondary_y=True, width=4, color='gray', opacity=0.2, shape='hv', connectgaps=True),
    Profit=dict(x=P.Index, y=P.Profit.cumsum(), secondary_y=True, width=6, color='blue', opacity=0.4, shape='hv', connectgaps=True)
)

fig.add_bar(name='Buy', x=Buy.Index, y=Buy.Volume, row=2, col=1, marker_color='green')
fig.add_bar(name='Sell', x=Sell.Index, y=Sell.Volume, row=2, col=1, marker_color='red')
fig.add_hline(y=threshold, row=4, col=1, line_dash='dot', line_color='gray')

# styling
fig.update_xaxes(spikemode='across+marker', spikedash='dot', spikethickness=2, spikesnap='cursor')
fig.update_traces(xaxis='x4')
fig.update_layout(legend=dict(x=0, y=1.05, bordercolor="lightgray", borderwidth=1))

fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.25,
   …